## GNN+WT-AWP for clean accuracy

In [1]:
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.utils import preprocess
import os
import torch
from deeprobust.graph.data import Dataset,Dpr2Pyg,Pyg2Dpr
from deeprobust.graph.defense import GCN, GAT, PPNP
from scipy import sparse
torch.cuda.set_device(3) 
device = torch.device("cuda")# if torch.cuda.is_available() else "cpu")


#### Train a GNN+WT-AWP model

In [2]:
def train_wtawp(data_name,model_name, seed_data,seed,weight,gamma):
    data = Dataset(root='/tmp/', name=data_name,setting = 'nettack',seed = seed_data)
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
    if model_name == "gcn":
        model = GCN(nfeat=features.shape[1], nclass=labels.max()+1, nhid=64, device=device)
        model = model.to(device)
        proxy = GCN(nfeat=features.shape[1], nclass=labels.max()+1, nhid=64, device=device)
        proxy = proxy.to(device)
        model.fit_mp(features, adj, labels, idx_train,proxy=proxy,idx_val=idx_val,verbose=False,
        awp_weight = weight, awp_gamma = gamma,train_iters = 200,idx_test = idx_test,seed=seed,awp_step = 1)
    elif model_name == "gat":
        pyg_data = Dpr2Pyg(data)
        model = GAT(nfeat=features.shape[1],
                          nhid=8, heads=8,
                          nclass=labels.max().item() + 1,
                          dropout=0.5, device=device)
        model = model.to(device)
         # convert deeprobust dataset to pyg dataset
        # gat.fit(pyg_data, patience=200, verbose=True) # train with earlystopping
        proxy = GAT(nfeat=features.shape[1],
                  nhid=8, heads=8,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)
        proxy = proxy.to(device)
        model.fit_mp(pyg_data,proxy=proxy,idx_val=idx_val,verbose=False,
                 awp_weight = weight, awp_gamma = gamma,train_iters = 200,awp_step=1,idx_test = idx_test,seed=seed)      
    elif model_name == "ppnp":
        pyg_data = Dpr2Pyg(data)
        model = PPNP(nfeat=features.shape[1],
                          nhid=64, 
                          nclass=labels.max().item() + 1,
                          dropout=0.5, device=device)
        model = model.to(device)
        proxy = PPNP(nfeat=features.shape[1],
                      nhid=64, 
                      nclass=labels.max().item() + 1,
                      dropout=0.5, device=device)
        proxy = proxy.to(device)
        model.fit_mp(pyg_data,proxy=proxy,idx_val=idx_val,verbose=False,
                     awp_weight = weight, awp_gamma = gamma,train_iters = 200,awp_step=1,idx_test = idx_test,seed=seed)       
    return model.test(idx_test)

In [15]:
train_wtawp(data_name = 'cora',model_name = 'ppnp',seed_data = 2,seed = 2, weight = 0.7,gamma=1)

Loading cora dataset...
Selecting 1 largest connected components
Processing...
Done!
Test set results: loss= 0.4327 accuracy= 0.8587


0.8586519114688129

#### Train a vanilla GNN model

In [6]:
def train_normal(data_name,model_name,seed_data,seed):
    data = Dataset(root='/tmp/', name=data_name,setting = 'nettack',seed = seed_data)
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
    if model_name == "gcn":
        model = GCN(nfeat=features.shape[1], nclass=labels.max()+1, nhid=64, device=device)
        model = model.to(device)
        model.fit(features, adj, labels, idx_train,idx_val,seed = seed,train_iters=200)
    elif model_name == "gat":
        # convert deeprobust dataset to pyg dataset
        pyg_data = Dpr2Pyg(data)
        model = GAT(nfeat=features.shape[1],
                          nhid=8, heads=8,
                          nclass=labels.max().item() + 1,
                          dropout=0.5, device=device)
        model = model.to(device)
        model.fit(pyg_data, patience=200, verbose=False,seed =seed) 
        
    elif model_name == "ppnp":
        # convert deeprobust dataset to pyg dataset
        pyg_data = Dpr2Pyg(data)
        model = PPNP(nfeat=features.shape[1],
                          nhid=64, 
                          nclass=labels.max().item() + 1,
                          dropout=0.5, device=device)
        model = model.to(device)
        model.fit(pyg_data, patience=200, verbose=False,seed =seed) 
    return model.test(idx_test)

In [16]:
train_normal(data_name = 'cora',model_name = 'ppnp',seed_data = 2, seed = 2)

Loading cora dataset...
Selecting 1 largest connected components
Processing...
Done!
Test set results: loss= 0.4756 accuracy= 0.8556


0.8556338028169015